<a href="https://colab.research.google.com/github/BeagleTamer100/256/blob/main/Clustering_Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install statsforecast
!pip install catboost
!pip install mlforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.7 MB/s eta 0:00:00


In [ ]:
# General-purpose libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, mean_squared_error
from sklearn.model_selection import ParameterGrid

# Forecasting libraries
from statsforecast.models import (
    SeasonalNaive, AutoTheta, CrostonClassic, CrostonOptimized,
    ADIDA, CrostonSBA, IMAPA, TSB
)
from statsforecast.core import StatsForecast
from mlforecast import MLForecast
from mlforecast.lag_transforms import RollingMean

# Machine learning regressors for forecasting
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# Set output directory
output_dir = 'Corona'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Load the data
df = pd.read_csv('Customer Order Quantity_Dispatched Quantity_cleaned.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')

In [ ]:
# Create complete date range for each product
def create_complete_timeseries(df):
    # Get min and max dates
    min_date = df['Date'].min()
    max_date = df['Date'].max()

    # Create complete date range
    date_range = pd.date_range(start=min_date, end=max_date, freq='D')

    # Get unique products
    products = df['Product ID'].unique()

    # Create empty list to store results
    complete_data = []

    # For each product, create complete time series
    for product in products:
        product_data = df[df['Product ID'] == product]

        # Create template with all dates
        template = pd.DataFrame(date_range, columns=['Date'])
        template['Product ID'] = product
        template['Product Name'] = product_data['Product Name'].iloc[0]

        # Merge with actual data
        merged = pd.merge(template, product_data[['Date', 'Customer Order Quantity']],
                         on='Date', how='left')

        # Fill NaN with 0
        merged['Customer Order Quantity'] = merged['Customer Order Quantity'].fillna(0)

        complete_data.append(merged)

    # Combine all products
    result = pd.concat(complete_data, ignore_index=True)
    return result


In [ ]:
# Calculate additional metrics for clustering
def calculate_metrics(df):
    metrics = []

    for product in df['Product ID'].unique():
        product_data = df[df['Product ID'] == product]
        demand = product_data['Customer Order Quantity'].values

        # Basic metrics
        avg_demand = np.mean(demand) + np.finfo(float).eps
        std_demand = np.std(demand)
        cv = std_demand / avg_demand

        # Calculate demand frequency
        non_zero_demands = demand[demand > 0]
        demand_frequency = len(non_zero_demands) / len(demand)

        # Calculate average order size when orders occur
        avg_order_size = np.mean(non_zero_demands) if len(non_zero_demands) > 0 else 0

        # Calculate peak-to-average ratio
        peak_demand = np.max(demand)
        peak_to_avg = peak_demand / avg_demand if avg_demand > 0 else 0

        # Calculate demand variability
        rolling_mean = pd.Series(demand).rolling(window=7).mean()
        demand_variability = np.std(rolling_mean) / np.mean(rolling_mean) if np.mean(rolling_mean) > 0 else 0

        # Calculate intervals between orders
        order_dates = np.where(demand > 0)[0]
        if len(order_dates) > 1:
            intervals = np.diff(order_dates)
            avg_interval = np.mean(intervals)
            interval_cv = np.std(intervals) / np.mean(intervals) if np.mean(intervals) > 0 else 0
        else:
            avg_interval = len(demand)
            interval_cv = 0

        zero_percentage = (demand == 0).mean() * 100

        metrics.append({
            'Product ID': product,
            'Product Name': product_data['Product Name'].iloc[0],
            'Average Demand': avg_demand,
            'CV': cv,
            'Average Interval': avg_interval,
            'Zero Percentage': zero_percentage,
            'Demand Frequency': demand_frequency,
            'Average Order Size': avg_order_size,
            'Peak to Average': peak_to_avg,
            'Demand Variability': demand_variability,
            'Interval CV': interval_cv
        })

    return pd.DataFrame(metrics)


In [ ]:
# Create complete timeseries and calculate metrics
complete_df = create_complete_timeseries(df)
metrics_df = calculate_metrics(complete_df)


# Modified try_clustering_methods function to provide more detailed results
def try_clustering_methods(X_scaled, metrics_df):
    results = {}

    # 1. K-means with different k
    print("\nTesting K-means clustering:")
    for k in range(3, 7):
        kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
        labels = kmeans.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        results[f'KMeans-{k}'] = (labels, score)
        print(f"K={k}: Silhouette Score = {score:.3f}, Cluster sizes = {np.bincount(labels)}")

    # 2. DBSCAN with different parameters
    print("\nTesting DBSCAN clustering:")
    for eps in [0.5, 0.7, 1.0]:
        dbscan = DBSCAN(eps=eps, min_samples=3)
        labels = dbscan.fit_predict(X_scaled)
        # Handle noise points (-1 labels) for visualization
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        if n_clusters > 1:  # Only calculate score if more than one cluster
            # Filter out noise points for silhouette score
            non_noise_mask = labels != -1
            score = silhouette_score(X_scaled[non_noise_mask], labels[non_noise_mask])
            results[f'DBSCAN-{eps}'] = (labels, score)
            # Count occurrences including noise points
            unique, counts = np.unique(labels, return_counts=True)
            cluster_sizes = dict(zip(unique, counts))
            print(f"eps={eps}: Silhouette Score = {score:.3f}")
            print(f"Cluster sizes (including noise points): {cluster_sizes}")

    # 3. Hierarchical clustering
    print("\nTesting Hierarchical clustering:")
    for k in range(3, 7):
        hc = AgglomerativeClustering(n_clusters=k)
        labels = hc.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        results[f'Hierarchical-{k}'] = (labels, score)
        print(f"K={k}: Silhouette Score = {score:.3f}, Cluster sizes = {np.bincount(labels)}")

    # Find best method
    best_method = max(results.items(), key=lambda x: x[1][1])
    print(f"\nBest clustering method: {best_method[0]}")
    print(f"Best silhouette score: {best_method[1][1]:.3f}")

    return best_method[1][0], results


# Prepare data for clustering using selected features
clustering_features = [
    'CV',
    'Average Interval',
    'Average Demand',
    'Average Order Size'
]


print("\nUsing clustering features:", clustering_features)


# Prepare the feature matrix
X = metrics_df[clustering_features].values


# Use RobustScaler for better handling of outliers
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)


# Print feature ranges after scaling
print("\nFeature ranges after scaling:")
for i, feature in enumerate(clustering_features):
    print(f"{feature}: {X_scaled[:, i].min():.2f} to {X_scaled[:, i].max():.2f}")


# Try different clustering methods and get best results
best_labels, all_results = try_clustering_methods(X_scaled, metrics_df)
metrics_df['Cluster'] = best_labels


# Print cluster characteristics
print("\nCluster Characteristics:")
cluster_stats = metrics_df.groupby('Cluster')[clustering_features].agg([
    ('mean', 'mean'),
    ('min', 'min'),
    ('max', 'max')
]).round(2)
print(cluster_stats)


# Create visualizations
plt.figure(figsize=(20, 15))


# Plot 1: CV vs Average Interval
plt.subplot(2, 2, 1)
scatter = plt.scatter(metrics_df['CV'], metrics_df['Average Interval'],
                     c=metrics_df['Cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Coefficient of Variation (CV)')
plt.ylabel('Average Interval (days)')
plt.title('CV vs Average Interval by Cluster')
plt.colorbar(scatter)


# Plot 2: Average Demand vs Average Order Size
plt.subplot(2, 2, 2)
scatter = plt.scatter(metrics_df['Average Demand'], metrics_df['Average Order Size'],
                     c=metrics_df['Cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Average Demand')
plt.ylabel('Average Order Size')
plt.title('Average Demand vs Order Size by Cluster')
plt.colorbar(scatter)


# Plot 3: CV vs Average Demand
plt.subplot(2, 2, 3)
scatter = plt.scatter(metrics_df['CV'], metrics_df['Average Demand'],
                     c=metrics_df['Cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Coefficient of Variation (CV)')
plt.ylabel('Average Demand')
plt.title('CV vs Average Demand')
plt.colorbar(scatter)


# Plot 4: Cluster Characteristics Heatmap
plt.subplot(2, 2, 4)
cluster_means = metrics_df.groupby('Cluster')[clustering_features].mean()
sns.heatmap(cluster_means, annot=True, cmap='YlOrRd', fmt='.2f')
plt.title('Cluster Characteristics')


plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'clustering_analysis.png'))
plt.close()


# Print detailed cluster descriptions
print("\nDetailed Cluster Descriptions:")
for cluster in sorted(metrics_df['Cluster'].unique()):
    cluster_data = metrics_df[metrics_df['Cluster'] == cluster]
    print(f"\nCluster {cluster} ({len(cluster_data)} products):")
    print(f"CV: {cluster_data['CV'].mean():.2f} (±{cluster_data['CV'].std():.2f})")
    print(f"Average Interval: {cluster_data['Average Interval'].mean():.2f} days (±{cluster_data['Average Interval'].std():.2f})")
    print(f"Average Demand: {cluster_data['Average Demand'].mean():.2f} (±{cluster_data['Average Demand'].std():.2f})")
    print(f"Average Order Size: {cluster_data['Average Order Size'].mean():.2f} (±{cluster_data['Average Order Size'].std():.2f})")
    print("Example products:", ", ".join(cluster_data['Product Name'].head(3).tolist()))


def analyze_best_clustering(metrics_df, best_labels, clustering_features):
    """Analyze and visualize the best clustering results in detail"""
    metrics_df['Cluster'] = best_labels

    # Create a directory for the analysis outputs
    analysis_dir = os.path.join(output_dir, 'cluster_analysis')
    if not os.path.exists(analysis_dir):
        os.makedirs(analysis_dir)

    # 1. Time Series Plot by Cluster
    plt.figure(figsize=(20, 12))
    for cluster in sorted(metrics_df['Cluster'].unique()):
        cluster_products = metrics_df[metrics_df['Cluster'] == cluster]['Product ID'].values
        cluster_data = complete_df[complete_df['Product ID'].isin(cluster_products)]

        # Calculate daily average demand for the cluster
        daily_demand = cluster_data.groupby('Date')['Customer Order Quantity'].mean()

        plt.plot(daily_demand.index, daily_demand.values,
                label=f'Cluster {cluster} (n={len(cluster_products)})')

    plt.title('Average Daily Demand by Cluster')
    plt.xlabel('Date')
    plt.ylabel('Average Demand')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(analysis_dir, 'demand_by_cluster.png'))
    plt.close()

    # 2. Cluster Characteristics Radar Chart
    plt.figure(figsize=(15, 15))

    # Normalize features for radar chart
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(metrics_df[clustering_features])
    normalized_df = pd.DataFrame(normalized_features, columns=clustering_features)
    normalized_df['Cluster'] = metrics_df['Cluster']

    # Calculate mean values for each cluster
    cluster_means = normalized_df.groupby('Cluster').mean()

    # Prepare radar chart
    angles = np.linspace(0, 2*np.pi, len(clustering_features), endpoint=False)
    angles = np.concatenate((angles, [angles[0]]))  # complete the circle

    ax = plt.subplot(111, projection='polar')

    for cluster in cluster_means.index:
        values = cluster_means.loc[cluster].values
        values = np.concatenate((values, [values[0]]))  # complete the circle
        ax.plot(angles, values, 'o-', linewidth=2, label=f'Cluster {cluster}')
        ax.fill(angles, values, alpha=0.25)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(clustering_features)
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title('Cluster Characteristics Radar Chart')
    plt.tight_layout()
    plt.savefig(os.path.join(analysis_dir, 'cluster_radar.png'))
    plt.close()

    # 3. Detailed Statistics Table
    detailed_stats = metrics_df.groupby('Cluster').agg({
        'Product ID': 'count',
        'Average Demand': ['mean', 'std', 'min', 'max'],
        'CV': ['mean', 'std'],
        'Zero Percentage': ['mean', 'std'],
        'Demand Frequency': ['mean', 'std'],
        'Average Order Size': ['mean', 'std'],
        'Peak to Average': ['mean', 'std']
    }).round(2)

    detailed_stats.to_csv(os.path.join(analysis_dir, 'detailed_cluster_stats.csv'))

    # 4. Box Plots for Key Metrics
    plt.figure(figsize=(20, 15))
    key_metrics = ['Average Demand', 'CV', 'Zero Percentage', 'Average Order Size']

    for i, metric in enumerate(key_metrics, 1):
        plt.subplot(2, 2, i)
        sns.boxplot(x='Cluster', y=metric, data=metrics_df)
        plt.title(f'{metric} Distribution by Cluster')
        plt.xticks(rotation=0)

    plt.tight_layout()
    plt.savefig(os.path.join(analysis_dir, 'cluster_boxplots.png'))
    plt.close()

    # 5. Scatter Matrix of Key Features
    plt.figure(figsize=(20, 20))
    scatter_features = ['Average Demand', 'CV', 'Zero Percentage', 'Average Order Size']
    sns.pairplot(metrics_df, vars=scatter_features, hue='Cluster', diag_kind='kde')
    plt.tight_layout()
    plt.savefig(os.path.join(analysis_dir, 'feature_scatter_matrix.png'))
    plt.close()

    # 6. Summary Report with Complete Product Lists
    with open(os.path.join(analysis_dir, 'cluster_summary.txt'), 'w') as f:
        f.write("Cluster Analysis Summary\n")
        f.write("=======================\n\n")

        for cluster in sorted(metrics_df['Cluster'].unique()):
            cluster_data = metrics_df[metrics_df['Cluster'] == cluster]
            f.write(f"\nCluster {cluster}:\n")
            f.write(f"Number of products: {len(cluster_data)}\n")
            f.write(f"Average demand: {cluster_data['Average Demand'].mean():.2f}\n")
            f.write(f"Average CV: {cluster_data['CV'].mean():.2f}\n")
            f.write(f"Average order frequency: {cluster_data['Demand Frequency'].mean():.2%}\n")

            f.write("\nKey Characteristics:\n")
            f.write(f"- Order size range: {cluster_data['Average Order Size'].min():.1f} - {cluster_data['Average Order Size'].max():.1f}\n")
            f.write(f"- Zero demand days: {cluster_data['Zero Percentage'].mean():.1f}%\n")
            f.write(f"- Peak to average ratio: {cluster_data['Peak to Average'].mean():.1f}\n")

            f.write("\nComplete Product List:\n")
            f.write("-" * 20 + "\n")
            # Sort products by average demand within cluster
            sorted_products = cluster_data.sort_values('Average Demand', ascending=False)
            for idx, row in sorted_products.iterrows():
                f.write(f"Product: {row['Product Name']}\n")
                f.write(f"  - Average Demand: {row['Average Demand']:.2f}\n")
                f.write(f"  - CV: {row['CV']:.2f}\n")
                f.write(f"  - Order Frequency: {row['Demand Frequency']:.2%}\n")
                f.write(f"  - Average Order Size: {row['Average Order Size']:.2f}\n")
                f.write("\n")

            f.write("\n" + "="*50 + "\n")


    # Add the new radar chart with the analysis_dir parameter
    create_detailed_radar_chart(metrics_df, clustering_features, analysis_dir)


def create_detailed_radar_chart(metrics_df, clustering_features, output_directory):
    """Create a radar chart showing mean, max, and min values for each cluster"""
    plt.figure(figsize=(20, 10))

    # Create two subplots side by side
    ax1 = plt.subplot(121, projection='polar')
    ax2 = plt.subplot(122, projection='polar')

    # 1. Normalize features
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(metrics_df[clustering_features])
    normalized_df = pd.DataFrame(normalized_features, columns=clustering_features)
    normalized_df['Cluster'] = metrics_df['Cluster']

    # 2. Calculate statistics for each cluster
    cluster_means = normalized_df.groupby('Cluster').mean()
    cluster_max = normalized_df.groupby('Cluster').max()
    cluster_min = normalized_df.groupby('Cluster').min()

    # 3. Prepare angles
    angles = np.linspace(0, 2*np.pi, len(clustering_features), endpoint=False)
    angles = np.concatenate((angles, [angles[0]]))  # complete the circle

    # Colors for each cluster
    colors = ['#2ecc71', '#e74c3c', '#3498db']

    # 4. Plot mean values (left subplot)
    for idx, cluster in enumerate(cluster_means.index):
        values = cluster_means.loc[cluster].values
        values = np.concatenate((values, [values[0]]))
        ax1.plot(angles, values, 'o-', linewidth=2, label=f'Cluster {cluster}', color=colors[idx])
        ax1.fill(angles, values, alpha=0.25, color=colors[idx])

    # Set chart attributes for mean values
    ax1.set_xticks(angles[:-1])
    ax1.set_xticklabels(clustering_features, size=8)
    ax1.set_title('Cluster Means')
    ax1.legend(loc='upper right', bbox_to_anchor=(0.3, 0.3))

    # 5. Plot mean, max, and min values (right subplot)
    line_styles = ['-', '--', ':']  # for mean, max, min

    for idx, cluster in enumerate(cluster_means.index):
        # Plot mean
        mean_values = cluster_means.loc[cluster].values
        mean_values = np.concatenate((mean_values, [mean_values[0]]))
        ax2.plot(angles, mean_values, 'o-', linewidth=2,
                label=f'Cluster {cluster} (Mean)', color=colors[idx])

        # Plot max
        max_values = cluster_max.loc[cluster].values
        max_values = np.concatenate((max_values, [max_values[0]]))
        ax2.plot(angles, max_values, 's--', linewidth=1,
                label=f'Cluster {cluster} (Max)', color=colors[idx])

        # Plot min
        min_values = cluster_min.loc[cluster].values
        min_values = np.concatenate((min_values, [min_values[0]]))
        ax2.plot(angles, min_values, '^:', linewidth=1,
                label=f'Cluster {cluster} (Min)', color=colors[idx])

        # Fill between min and max
        ax2.fill_between(angles, min_values, max_values, alpha=0.1, color=colors[idx])

    # Set chart attributes for detailed values
    ax2.set_xticks(angles[:-1])
    ax2.set_xticklabels(clustering_features, size=8)
    ax2.set_title('Cluster Means with Min-Max Ranges')
    ax2.legend(loc='upper right', bbox_to_anchor=(0.3, 0.3))

    plt.tight_layout()
    plt.savefig(os.path.join(output_directory, 'detailed_cluster_radar.png'),
                bbox_inches='tight', dpi=300)
    plt.close()


# After getting best_labels, call the analysis function
analyze_best_clustering(metrics_df, best_labels, clustering_features)


# Create demand pattern classification plot
plt.figure(figsize=(12, 8))


# Standard thresholds
cv2_threshold = 0.49  # Square of 0.7
adi_threshold = 1.32


plt.axhline(y=adi_threshold, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=cv2_threshold, color='gray', linestyle='--', alpha=0.3)


# Add region labels
plt.text(0.25, 0.5, 'Smooth', ha='center', va='center', alpha=0.3)
plt.text(1.0, 0.5, 'Erratic', ha='center', va='center', alpha=0.3)
plt.text(0.25, 2, 'Intermittent', ha='center', va='center', alpha=0.3)
plt.text(1.0, 2, 'Lumpy', ha='center', va='center', alpha=0.3)


# Calculate CV squared for plotting
cv_squared = metrics_df['CV'] ** 2


# Create scatter plot
scatter = plt.scatter(cv_squared,
                     metrics_df['Average Interval'],
                     c=metrics_df['Cluster'],
                     cmap='viridis',
                     alpha=0.6)


plt.xlabel('Coefficient of Variation Squared (CV²)')
plt.ylabel('Average Interval (days)')
plt.title('Demand Pattern Classification')
plt.colorbar(scatter, label='Cluster')


# Set axis limits based on data with padding
x_max = max(2, cv_squared.max() * 1.1)
y_max = max(3, metrics_df['Average Interval'].max() * 1.1)
plt.xlim(-0.1, x_max)
plt.ylim(-0.1, y_max)


plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'demand_pattern_classification.png'))
plt.close()


# Create final summary with pattern classifications
def classify_pattern(cv, interval):
    """
    Classify demand pattern based on CV² and ADI thresholds
    CV² threshold = 0.49
    ADI threshold = 1.32
    """
    cv_squared = cv ** 2
    if cv_squared <= 0.49:
        if interval <= 1.32:
            return 'Smooth'
        else:
            return 'Intermittent'
    else:
        if interval <= 1.32:
            return 'Erratic'
        else:
            return 'Lumpy'


# Create final summary DataFrame
final_summary = pd.DataFrame({
    'Product ID': metrics_df['Product ID'],
    'Product Name': metrics_df['Product Name'],
    'Cluster': metrics_df['Cluster'],
    'CV': metrics_df['CV'],
    'CV_Squared': metrics_df['CV'] ** 2,
    'Average Interval': metrics_df['Average Interval'],
    'Average Demand': metrics_df['Average Demand'],
    'Average Order Size': metrics_df['Average Order Size']
})


# Add pattern classification
final_summary['Pattern'] = final_summary.apply(
    lambda row: classify_pattern(row['CV'], row['Average Interval']),
    axis=1
)


# Save final summary
final_summary.to_csv(os.path.join(output_dir, 'final_pattern_summary.csv'), index=False)


# Print summary statistics
print("\nDemand Pattern Distribution:")
pattern_counts = final_summary['Pattern'].value_counts()
for pattern, count in pattern_counts.items():
    print(f"{pattern}: {count} products ({count/len(final_summary)*100:.1f}%)")


print("\nCluster and Pattern Cross-tabulation:")
cross_tab = pd.crosstab(final_summary['Cluster'], final_summary['Pattern'])
print(cross_tab)



Using clustering features: ['CV', 'Average Interval', 'Average Demand', 'Average Order Size']

Feature ranges after scaling:
CV: -1.17 to 27.31
Average Interval: -0.71 to 52.04
Average Demand: -0.44 to 32.75
Average Order Size: -0.49 to 81.34

Testing K-means clustering:
K=3: Silhouette Score = 0.848, Cluster sizes = [ 5  1 78]
K=4: Silhouette Score = 0.828, Cluster sizes = [78  1  4  1]
K=5: Silhouette Score = 0.756, Cluster sizes = [74  1  2  1  6]
K=6: Silhouette Score = 0.737, Cluster sizes = [74  1  1  1  5  2]

Testing DBSCAN clustering:
eps=0.5: Silhouette Score = 0.270
Cluster sizes (including noise points): {-1: 23, 0: 48, 1: 5, 2: 3, 3: 5}

Testing Hierarchical clustering:
K=3: Silhouette Score = 0.848, Cluster sizes = [ 5  1 78]
K=4: Silhouette Score = 0.828, Cluster sizes = [78  1  4  1]
K=5: Silhouette Score = 0.661, Cluster sizes = [ 4  7 71  1  1]
K=6: Silhouette Score = 0.663, Cluster sizes = [ 3  7 71  1  1  1]

Best clustering method: KMeans-3
Best silhouette score: 

<Figure size 2000x2000 with 0 Axes>

In [ ]:
# Prepare data for forecasting
weekly_df = df.groupby(['Date', 'Product ID'])['Customer Order Quantity'].sum().reset_index()
weekly_df.columns = ["ds", "unique_id", "y"]
weekly_df["ds"] = pd.to_datetime(weekly_df["ds"])

# Split into train and test
train = weekly_df[weekly_df.ds <= '2024-06-30']
test = weekly_df[weekly_df.ds > '2024-06-30']

# Save test data
test.to_csv(os.path.join(output_dir, 'test_data.csv'), index=False)

# Define forecast settings
season_length = 24
horizon = len(test['ds'].unique())
freq = "W-MON"

# Define forecasting models
models = [
    SeasonalNaive(season_length=season_length),
    AutoTheta(season_length=season_length, decomposition_type="additive", model="STM"),
    CrostonClassic(),
    CrostonOptimized(),
    ADIDA(),
    CrostonSBA(),
    IMAPA(),
    TSB(alpha_d=0.5, alpha_p=0.5)
]

sf = StatsForecast(models=models, freq=freq, n_jobs=-1)

# Generate forecasts
forecasts_df = sf.forecast(df=train, h=horizon)

# Additional forecasting methods
def moving_average_forecast(df, window=4, horizon=4):
    forecasts = []
    for uid in df["unique_id"].unique():
        series = df[df["unique_id"] == uid].sort_values("ds")
        past_values = series["y"].rolling(window=window).mean().iloc[-1] if len(series) >= window else series["y"].mean()
        future_dates = pd.date_range(start=series["ds"].max(), periods=horizon + 1, freq="W-MON")[1:]
        forecast_df = pd.DataFrame({"unique_id": uid, "ds": future_dates, "moving_avg_forecast": [past_values] * horizon})
        forecasts.append(forecast_df)
    return pd.concat(forecasts, ignore_index=True)

def weighted_moving_average_forecast(df, window=4, horizon=4):
    forecasts = []
    weights = np.arange(1, window + 1) / sum(np.arange(1, window + 1))
    for uid in df["unique_id"].unique():
        series = df[df["unique_id"] == uid].sort_values("ds")
        if len(series) >= window:
            past_values = np.dot(series["y"].iloc[-window:], weights)
        else:
            past_values = series["y"].mean()
        future_dates = pd.date_range(start=series["ds"].max(), periods=horizon + 1, freq="W-MON")[1:]
        forecast_df = pd.DataFrame({"unique_id": uid, "ds": future_dates, "wma_forecast": [past_values] * horizon})
        forecasts.append(forecast_df)
    return pd.concat(forecasts, ignore_index=True)

def exponential_smoothing_forecast(df, alpha=0.3, horizon=4):
    forecasts = []
    for uid in df["unique_id"].unique():
        series = df[df["unique_id"] == uid].sort_values("ds")["y"]
        if len(series) > 1:
            ses_value = series.iloc[0]
            for val in series:
                ses_value = alpha * val + (1 - alpha) * ses_value
        else:
            ses_value = series.mean()
        future_dates = pd.date_range(start=df[df["unique_id"] == uid]["ds"].max(), periods=horizon + 1, freq="W-MON")[1:]
        forecast_df = pd.DataFrame({"unique_id": uid, "ds": future_dates, "ses_forecast": [ses_value] * horizon})
        forecasts.append(forecast_df)
    return pd.concat(forecasts, ignore_index=True)

# Generate additional forecasts
moving_avg_df = moving_average_forecast(train, window=4, horizon=horizon)
wma_df = weighted_moving_average_forecast(train, window=4, horizon=horizon)
ses_df = exponential_smoothing_forecast(train, alpha=0.3, horizon=horizon)

# Merge all forecasts
forecasts_df = forecasts_df.merge(moving_avg_df, on=['unique_id', 'ds'], how='left')
forecasts_df = forecasts_df.merge(wma_df, on=['unique_id', 'ds'], how='left')
forecasts_df = forecasts_df.merge(ses_df, on=['unique_id', 'ds'], how='left')

# Save forecasts
forecasts_df.to_csv(os.path.join(output_dir, "statistical_full_data_forecast_with_MA_WMA_SES.csv"), index=False)

print("All necessary files have been created in the 'Corona' directory.")


All necessary files have been created in the 'Corona' directory.


In [ ]:
# Load the necessary data
metrics = pd.read_csv(os.path.join(output_dir, 'final_pattern_summary.csv'))
forecasts_df = pd.read_csv(os.path.join(output_dir, 'statistical_full_data_forecast_with_MA_WMA_SES.csv'))
test = pd.read_csv(os.path.join(output_dir, 'test_data.csv'))

# Merge cluster assignments with forecasts and test data
forecasts_df = forecasts_df.merge(metrics[['Product ID', 'Cluster']], left_on='unique_id', right_on='Product ID', how='left')
test = test.merge(metrics[['Product ID', 'Cluster']], left_on='unique_id', right_on='Product ID', how='left')

def calculate_cluster_rmse(test, forecasts_df):
    merged_df = pd.merge(test, forecasts_df, on=['unique_id', 'ds', 'Cluster'])
    rmse_scores = []

    model_columns = [col for col in forecasts_df.columns if col not in ['unique_id', 'ds', 'Cluster', 'Product ID']]

    for model in model_columns:
        cluster_rmse = merged_df.groupby('Cluster').apply(lambda x: np.sqrt(np.mean((x['y'] - x[model])**2)), include_groups=False)
        rmse_scores.append(pd.DataFrame({'Cluster': cluster_rmse.index, 'model': model, 'rmse': cluster_rmse.values}))

    return pd.concat(rmse_scores, ignore_index=True)

cluster_rmse_df = calculate_cluster_rmse(test, forecasts_df)


# Select the best model per cluster
def select_best_model_per_cluster(metrics_df):
    return metrics_df.loc[metrics_df.groupby('Cluster')['rmse'].idxmin()][['Cluster', 'model', 'rmse']]

best_models_per_cluster = select_best_model_per_cluster(cluster_rmse_df)

# Calculate average RMSE for each cluster using all products and their RMSEs
average_rmse_per_cluster = cluster_rmse_df.groupby('Cluster')['rmse'].mean().reset_index()
average_rmse_per_cluster.columns = ['Cluster', 'Average RMSE']

# Combine best models and average RMSE
final_results = best_models_per_cluster.merge(average_rmse_per_cluster, on='Cluster')


# Save the results
final_results.to_csv(os.path.join(output_dir, "best_forecasting_models_per_cluster.csv"), index=False)

print(final_results)


   Cluster           model        rmse  Average RMSE
0        0   SeasonalNaive    1.732051     81.753186
1        1             TSB  925.895152   1082.543126
2        2  CrostonClassic   41.534753     47.028832


In [ ]:
metrics_df

,Product ID,Product Name,Average Demand,CV,Average Interval,Zero Percentage,Demand Frequency,Average Order Size,Peak to Average,Demand Variability,Interval CV,Cluster
0,000161032,Material_4,47.747191,1.356218,1.345598,28.651685,0.713483,66.921260,9.424638,0.482497,0.591513,2
1,000179754,Material_12,4.817416,4.271823,3.720000,74.157303,0.258427,18.641304,112.508455,1.567808,0.847821,2
2,000179758,Material_14,0.547753,5.862913,16.098361,94.194757,0.058052,9.435484,109.538462,2.178872,0.944663,2
3,000180631,Material_17,4.548689,2.367486,2.672727,63.857678,0.361423,12.585492,27.480445,0.863758,0.913448,2
4,000180632,Material_19,20.960674,1.466526,1.476395,34.456929,0.655431,31.980000,12.404181,0.573975,0.616110,2
...,...,...,...,...,...,...,...,...,...,...,...,...
79,5551O1476,Material_62,5.140449,2.444614,2.915254,66.760300,0.332397,15.464789,54.859016,0.877297,0.575219,2
80,5551O2494,Material_73,5.742509,3.054945,2.826087,65.449438,0.345506,16.620596,64.780042,1.201124,0.807161,2
81,5551O2515,Material_75,0.253745,21.855419,21.368421,98.127341,0.018727,13.550000,693.608856,8.270066,1.354227,0
82,5551O2518,Material_76,23.842697,2.732534,2.400000,59.644195,0.403558,59.081206,70.461828,1.038591,0.928483,2
